# Imports

In [11]:
import numpy as np
import tensorflow as tf
import sys
sys.path.append('../../')
import utils
import custom_metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
tfk = tf.keras 
tfkl = tfk.layers

# Setting Seed

In [12]:
seed = 17560

np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.compat.v1.set_random_seed(seed)

# Model Building

In [13]:
training_dir = os.path.join('.', 'training')
validation_dir = os.path.join('.', 'validation')

batch_size = 64
epochs = 1000
patience = 20
hiddens = 2 
neurons = [256, 128]
input_shape = (256, 256, 3)

train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=45,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2)

valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=45,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2)

train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=input_shape[:2],
                                               interpolation='bilinear',
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               classes=None,
                                               shuffle=True,
                                               seed=seed)  

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=input_shape[:2],
                                               interpolation='bilinear',
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               classes=None,
                                               shuffle=False,
                                               seed=seed)  
                                        

Found 14176 images belonging to 14 classes.
Found 3552 images belonging to 14 classes.


In [14]:
classes_weights = wgt.compute_class_weight(class_weight='balanced', classes=np.unique(train_gen.classes), y=train_gen.classes)
classes_weights_map = {i: classes_weights[i] for i in range(0, len(classes_weights))}
print(classes_weights_map)

{0: 1.2817359855334538, 1: 2.714668709306779, 2: 2.1729000613120784, 3: 1.050385299347955, 4: 0.8684146042636609, 5: 0.7243000204373595, 6: 1.2965063105908177, 7: 1.6545284780578897, 8: 1.7702297702297702, 9: 4.7989167230873395, 10: 0.7837240159221583, 11: 2.206037970743853, 12: 1.8821030270844397, 13: 0.22234770060857018}


In [15]:
tl_model = utils.build_tl_vgg_model(hiddens=hiddens, neurons=neurons, input_shape=input_shape, seed=seed)

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=utils.metrics())
tl_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
Flattening (Flatten)         (None, 32768)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               8388864   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896 

# Training and Results Visualization

In [ ]:
tl_history = tl_model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = valid_gen,
    class_weight=classes_weights_map,
    callbacks = utils.callbacks("val_accuracy", "max", patience, 'sklearn_weights_cat_crossentropy_loss_model')
).history

In [ ]:
name = 'sklearn_weights_crossentropy_' + utils.get_model_name(hiddens=hiddens, neurons=neurons)
tl_model.save(name)

In [ ]:
utils.plot_history(tl_history)